In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Eur_Usd'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
df['ema500'] = pd.Series.ewm(df['Mid_Close'], span=500).mean()
df['ema400'] = pd.Series.ewm(df['Mid_Close'], span=400).mean()
df['ema300'] = pd.Series.ewm(df['Mid_Close'], span=300).mean()
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema500,ema400,ema300,ema200,ema100,ema50
0,2021-04-01 06:00:00,1.17170,1.17172,1.17141,1.17141,1.17184,1.17187,1.17154,1.17154,1.17177,1.17180,1.17148,1.17148,348,1.171480,1.171480,1.171480,1.171480,1.171480,1.171480
1,2021-04-01 06:05:00,1.17140,1.17182,1.17119,1.17154,1.17153,1.17194,1.17133,1.17167,1.17146,1.17188,1.17126,1.17160,451,1.171540,1.171540,1.171540,1.171540,1.171541,1.171541
2,2021-04-01 06:10:00,1.17153,1.17162,1.17140,1.17161,1.17166,1.17175,1.17151,1.17174,1.17160,1.17168,1.17146,1.17168,324,1.171587,1.171587,1.171587,1.171587,1.171588,1.171589
3,2021-04-01 06:15:00,1.17160,1.17187,1.17153,1.17185,1.17173,1.17200,1.17165,1.17197,1.17166,1.17194,1.17160,1.17191,275,1.171668,1.171668,1.171669,1.171669,1.171671,1.171674
4,2021-04-01 06:20:00,1.17184,1.17253,1.17171,1.17232,1.17196,1.17266,1.17185,1.17244,1.17190,1.17259,1.17178,1.17238,646,1.171812,1.171812,1.171813,1.171814,1.171818,1.171827


In [7]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema500,ema400,ema300,ema200,ema100,ema50
74778,2022-04-01 05:35:00,1.10690,1.10694,1.10668,1.10676,1.10707,1.10710,1.10683,1.10691,1.10698,1.10702,1.10676,1.10684,291,1.109588,1.109400,1.108874,1.107953,1.106952,1.106711
74779,2022-04-01 05:40:00,1.10675,1.10727,1.10672,1.10720,1.10691,1.10744,1.10689,1.10737,1.10683,1.10735,1.10680,1.10728,400,1.109579,1.109389,1.108863,1.107946,1.106958,1.106733
74780,2022-04-01 05:45:00,1.10721,1.10722,1.10698,1.10704,1.10737,1.10738,1.10715,1.10721,1.10729,1.10730,1.10707,1.10712,183,1.109569,1.109378,1.108851,1.107938,1.106961,1.106749
74781,2022-04-01 05:50:00,1.10703,1.10729,1.10700,1.10726,1.10721,1.10744,1.10717,1.10743,1.10712,1.10736,1.10708,1.10734,227,1.109561,1.109368,1.108841,1.107932,1.106969,1.106772
74782,2022-04-01 05:55:00,1.10727,1.10737,1.10667,1.10673,1.10742,1.10754,1.10683,1.10690,1.10734,1.10745,1.10675,1.10682,439,1.109550,1.109355,1.108828,1.107921,1.106966,1.106774


In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [9]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100
n_recent_trades = 2
recent_hours = 1

In [10]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, ema, invert):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk
    waiting_idx, prev_trade_date = 0, None

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]

        if not use_friday and curr_date.day == 4 and curr_date.hour >= 16:
            continue

        elif prev_trade_date is not None:
            diff = curr_date - prev_trade_date
            n_hours = (diff.total_seconds() / 60) / 60

            if waiting_idx >= n_recent_trades:
                if n_hours >= wait_period:
                    prev_trade_date, waiting_idx = None, 0

                else:
                    continue
        
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        if each_bar:
            buy_signal = all([mid_opens[j] < mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])
            sell_signal = all([mid_opens[j] > mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])

        else:
            buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
            sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal and not invert:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal and not invert:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if ema is not None:
            prev_ema, prev_mid_close = df.loc[df.index[i - 1], [ema, 'Mid_Close']]
            
            buy_signal = buy_signal and prev_mid_close > prev_ema
            sell_signal = sell_signal and prev_mid_close < prev_ema

        if invert:
            buy_signal, sell_signal = sell_signal, buy_signal

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        placed_trade = False

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

                        placed_trade = True

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

                        placed_trade = True


        if placed_trade and prev_trade_date is not None:
            diff = curr_date - prev_trade_date
            n_hours = (diff.total_seconds() / 60) / 60

            if n_hours <= recent_hours:
                waiting_idx += 1

        closed_trade = False

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None
            closed_trade = True

        if closed_trade:
            prev_trade_date = curr_date

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5]
spread_cutoffs = [0.10]
each_bar_vals = [False]
n_bars_vals = [3, 4, 5]
pip_movement_vals = [20, 30]
use_pullback_vals = [True]
use_friday_vals = [True, False]  # TRUE MEANS IT WILL TRADE ALL HOURS ON FRIDAY, FALSE MEANS IT WILL STOP AFTER 9 AM PST
wait_periods = [0, 1, 2, 3, 4]  # THESE ARE IN HOURS
pips_to_risk_vals = ['bars', 10, 15, 20]  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.10, 0.20, 0.25, 0.30, 0.40, 0.50]
ema_vals = [None, 'ema500', 'ema400', 'ema300', 'ema200', 'ema100', 'ema50']  # "None" MEANS IT WON'T USE THE EMA
invert_vals = [True, False]  # "False" MEANS IT WILL USE THE DEFAULT BEHAVIOR

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(each_bar_vals) * len(n_bars_vals) * len(pip_movement_vals) * len(use_pullback_vals) * len(use_friday_vals) * len(wait_periods) * len(pips_to_risk_vals) * len(pullback_percentages) * len(ema_vals) * len(invert_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for each_bar in each_bar_vals:
            for n_bars in n_bars_vals:
                for pip_movement in pip_movement_vals:
                    for use_pullback in use_pullback_vals:
                        for use_friday in use_friday_vals:
                            for wait_period in wait_periods:
                                for pips_to_risk in pips_to_risk_vals:
                                    for pullback_percentage in pullback_percentages:
                                        for ema in ema_vals:
                                            for invert in invert_vals:
                                                all_combos.append((risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, ema, invert))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_each_bar_val = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_use_friday_val = None
best_wait_period = None
best_pips_to_risk = None
best_pullback_percentage = None
best_ema = None
best_invert_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, ema, invert in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, ema, invert)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'each_bar': each_bar, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'use_friday': use_friday, 'wait_period': wait_period, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'ema': ema, 'invert': invert})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_each_bar_val = each_bar
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_use_friday_val = use_friday
        best_wait_period = wait_period
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_ema = ema
        best_invert_val = invert

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 40320

0.0010000000000047748
Num buys: 1
Num sells: 1
Num trades: 2
Num wins: 1
Num losses: 1
Win streak: 1
Loss streak: 1
Avg pips risked: 0.0010000000000000009
Remaining runs: 40319
Best reward so far: 0.0010000000000047748

349.98600000000033
Num buys: 5
Num sells: 6
Num trades: 11
Num wins: 9
Num losses: 2
Win streak: 7
Loss streak: 1
Avg pips risked: 0.0020000000000000018
Remaining runs: 40318
Best reward so far: 349.98600000000033



KeyboardInterrupt: 

In [12]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best use friday val: ' + str(best_use_friday_val))
print('Best wait period: ' + str(best_wait_period))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best ema: ' + str(best_ema))
print('Best invert val: ' + str(best_invert_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 349.98600000000033
Best risk/reward ratio: 1.0
Best spread: 0.1
Best each bar val: False
Best n bars val: 5
Best pip movement val: 30
Best use pullback val: True
Best use friday val: True
Best wait period: 2
Best pips to risk: 20
Best pullback percentage: 0.3
Best ema: ema400
Best invert val: False
-----------------------
Top results:
{'reward': 0, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True, 'use_friday': True, 'wait_period': 4, 'pips_to_risk': 10, 'pullback_percentage': 0.5, 'ema': None, 'invert': True}
{'reward': 349, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 30, 'use_pullback': True, 'use_friday': True, 'wait_period': 2, 'pips_to_risk': 20, 'pullback_percentage': 0.3, 'ema': 'ema400', 'invert': False}
